In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python.keras import models, layers, optimizers

# from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
# from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

%matplotlib inline


In [0]:
import tensorflow

In [0]:
from tensorflow.python.keras import models, layers, optimizers

In [0]:
from tensorflow.python.keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [0]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import bz2

In [0]:
path = '/content/drive/My Drive/train/train.csv'
df = pd.read_csv(path)

In [29]:
df.shape

(79979, 2)

In [13]:
df.isna().sum()

review_clean    21
sentiment        0
dtype: int64

In [0]:
df.dropna(inplace=True)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_data, test_data = train_test_split(df, test_size=0.2,random_state = 42)

In [0]:
train_labels = train_data.sentiment
train_texts = train_data.review_clean

test_labels = test_data.sentiment
test_texts = test_data.review_clean

In [0]:
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [0]:
MAX_FEATURES = 12000

tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train_texts)
train_texts = tokenizer.texts_to_sequences(train_texts)
test_texts = tokenizer.texts_to_sequences(test_texts)

In [0]:
MAX_LENGTH = max(len(train_ex) for train_ex in train_texts)
train_texts = pad_sequences(train_texts, maxlen=MAX_LENGTH)
test_texts = pad_sequences(test_texts, maxlen=MAX_LENGTH)

In [0]:
def build_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(MAX_FEATURES, 64)(sequences)
    x = layers.Conv1D(64, 3, activation='relu')(embedded)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(3)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
model = build_model()

In [22]:
model.fit(
    train_texts, 
    train_labels, 
    batch_size=128,
    epochs=2,
    validation_data=(test_texts, test_labels) )

Epoch 1/2
500/500 [==============================] - 56s 112ms/step - loss: 0.2157 - binary_accuracy: 0.9233 - val_loss: 0.1995 - val_binary_accuracy: 0.9208
Epoch 2/2
500/500 [==============================] - 55s 111ms/step - loss: 0.1456 - binary_accuracy: 0.9436 - val_loss: 0.1830 - val_binary_accuracy: 0.9384


In [0]:
def build_rnn_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(MAX_FEATURES, 64)(sequences)
    x = layers.CuDNNGRU(128, return_sequences=True)(embedded)
    x = layers.CuDNNGRU(128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
rnn_model = build_rnn_model()

In [28]:
rnn_model.fit(
    train_texts, 
    train_labels, 
    batch_size=128,
    epochs=2,
    validation_data=(test_texts, test_labels) )

Epoch 1/2
500/500 [==============================] - 331s 661ms/step - loss: 0.2204 - binary_accuracy: 0.9177 - val_loss: 0.1871 - val_binary_accuracy: 0.9280
Epoch 2/2
500/500 [==============================] - 329s 658ms/step - loss: 0.1611 - binary_accuracy: 0.9361 - val_loss: 0.1792 - val_binary_accuracy: 0.9369


In [0]:
def build_lstm_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(MAX_FEATURES, 64)(sequences)
    x = layers.LSTM(128, return_sequences=True)(embedded)
    x = layers.LSTM(128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
rnn_model = build_lstm_model()

In [26]:
rnn_model.fit(
    train_texts, 
    train_labels, 
    batch_size=128,
    epochs=2,
    validation_data=(test_texts, test_labels) )

Epoch 1/2
500/500 [==============================] - 392s 783ms/step - loss: 0.2149 - binary_accuracy: 0.9245 - val_loss: 0.1718 - val_binary_accuracy: 0.9322
Epoch 2/2
500/500 [==============================] - 391s 781ms/step - loss: 0.1652 - binary_accuracy: 0.9367 - val_loss: 0.1656 - val_binary_accuracy: 0.9375


In [31]:
len(train_texts)

63983

In [32]:
len(test_texts)

15996